In [1]:
import os

base_folder = r'C:\Users\Алексей\Documents\ETHZ\interhand_masked\masked\test\Capture0\ROM04_LT_Occlusion'
target_folder = r'C:\Users\Алексей\Documents\ETHZ\interhand_masked\single_image_per_camera'
target_folder_gifs = r'C:\Users\Алексей\Documents\ETHZ\interhand_masked\gifs'

target_folder_noised = r'C:\Users\Алексей\Documents\ETHZ\interhand_masked\single_image_per_camera_noised'

# make target folder if it doesn't exist
if not os.path.exists(target_folder):
    os.makedirs(target_folder)

if not os.path.exists(target_folder_gifs):
    os.makedirs(target_folder_gifs)

if not os.path.exists(target_folder_noised):
    os.makedirs(target_folder_noised)

In [2]:
import shutil

for folder in os.listdir(base_folder):
    images_list = sorted(os.listdir(os.path.join(base_folder, folder)))

    # copy the first image to the target folder, with name of the folder
    shutil.copy(os.path.join(base_folder, folder, images_list[0]), os.path.join(target_folder, folder + '.jpg'))

In [24]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

cameras_list = [
    'cam400263',
    'cam400293',
    'cam400296',
]





# for each camera, go to base_folder/camera_name, get first 50 images in that folder, add gaussian noise to them, save them to target_folder_noised/camera_name
import numpy as np
import cv2

for camera in cameras_list:
    images_list = sorted(os.listdir(os.path.join(base_folder, camera)))

    cam_dir = os.path.join(target_folder_noised, camera)
    if not os.path.exists(cam_dir):
        os.makedirs(cam_dir)

    for i in range(50):
        # read image without opencv
        image = plt.imread(os.path.join(base_folder, camera, images_list[i])) * 255


        # image = image + np.random.normal(0, 10, image.shape)
        # distort image
        image = cv2.resize(image, (0, 0), fx=0.85, fy=0.85)
        image = cv2.resize(image, (0, 0), fx=1.15, fy=1.15)


        image = np.clip(image, 0, 255)
        image = image.astype(np.uint8)

        # save image without opencv
        plt.imsave(os.path.join(cam_dir, images_list[i]), image)

In [8]:
from PIL import Image
from PIL import ImageDraw, ImageFont
import numpy as np
import cv2


def preprocess_frame(frame):

    # replace white with black
    frame_np = np.array(frame)
    frame_np[frame_np == 255] = 0

    mask = frame_np != 0

    frame_np_distorted = frame_np.copy()

    # add gaussian noise
    frame_np_distorted = frame_np_distorted + np.random.normal(0, 9, frame_np_distorted.shape)

    # multiply by mask
    frame_np_distorted = frame_np_distorted * mask

    compression_factor = 0.8
    frame_np_distorted = cv2.resize(frame_np_distorted, (0, 0), fx=compression_factor, fy=compression_factor)
    frame_np_distorted = cv2.resize(frame_np_distorted, (0, 0), fx=1/compression_factor, fy=1/compression_factor)

    # apply gaussian blur
    frame_np_distorted = cv2.GaussianBlur(frame_np_distorted, (5, 5), 0)

    frame_np_distorted = np.clip(frame_np_distorted, 0, 255)
    frame_np_distorted = frame_np_distorted.astype(np.uint8)

    frame = Image.fromarray(frame_np)
    I1 = ImageDraw.Draw(frame)
    font = ImageFont.truetype("arial.ttf", 30)
    I1.text((100, frame_np.shape[0] - 40), "Ground truth", fill=(254, 254, 254), font=font)

    frame_dist = Image.fromarray(frame_np_distorted)
    I2 = ImageDraw.Draw(frame_dist)
    I2.text((100, frame_np.shape[0] - 40), "Predicted", fill=(254, 254, 254), font=font)

    # make new frame of size 2 * width
    new_frame = Image.new('RGB', (frame.width * 2, frame.height))
    new_frame.paste(frame, (0, 0))
    new_frame.paste(frame_dist, (frame.width, 0))

    return new_frame

    # return frame


def make_gif(frame_list, gif_name, duration=100):
    frames = [Image.open(image) for image in frame_list]

    for i in range(len(frames)):
        frames[i] = preprocess_frame(frames[i])

    frame_one = frames[0]
    frame_one.save(gif_name, format="GIF", append_images=frames,
               save_all=True, duration=duration, loop=0)


cameras_list = [
    'cam400263',
    'cam400280',
    'cam400293',
    'cam400296',
]

for camera in cameras_list:
    images_list = sorted(os.listdir(os.path.join(base_folder, camera)))
    full_images_list = [os.path.join(base_folder, camera, image) for image in images_list[:50]]

    make_gif(full_images_list,
             os.path.join(target_folder_gifs, camera + '.gif'), duration=250)